In [4]:
#move all files from FLIR/train/thermal_8_bit to FLIR/train
import os
import shutil

src = 'FLIR/train/thermal_8_bit'
dst = 'FLIR/train'

for file in os.listdir(src):
    shutil.move(os.path.join(src, file), dst)


In [8]:
!python3 -m pip install -r requirements.txt

     |████████████████████████████████| 172 kB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 59 kB 1.3 MB/s  eta 0:00:01
     |████████████████████████████████| 52 kB 159 kB/s  eta 0:00:01
     |████████████████████████████████| 11.1 MB 12.0 MB/s eta 0:00:01
     |████████████████████████████████| 119 kB 12.8 MB/s eta 0:00:01
     |████████████████████████████████| 13.9 MB 10.7 MB/s eta 0:00:01
     |████████████████████████████████| 5.5 MB 8.4 MB/s eta 0:00:01
     |████████████████████████████████| 256 kB 11.8 MB/s eta 0:00:01
     |████████████████████████████████| 3.7 MB 8.5 MB/s eta 0:00:01
     |████████████████████████████████| 177 kB 11.7 MB/s eta 0:00:01
     |████████████████████████████████| 301 kB 11.0 MB/s eta 0:00:01
     |████████████████████████████████| 220 kB 11.4 MB/s eta 0:00:01
     |████████████████████████████████| 315 kB 10.7 MB/s eta 0:00:01
     |████████████████████████████████| 6.9 MB 9.3 MB/s eta 0:00:01
     |████████████████████████████████

In [22]:
#Functions to convert COCO Json to YOLO txt
from glob import glob
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
from tqdm import tqdm

def make_folders(output_path):
    #if os.path.exists(output_path):
        #shutil.rmtree(output_path)
    os.makedirs(output_path)
    return output_path


def convert_bbox_coco2yolo(img_width, img_height, bbox):
    """
    Convert bounding box from COCO  format to YOLO format

    Parameters
    ----------
    img_width : int
        width of image
    img_height : int
        height of image
    bbox : list[int]
        bounding box annotation in COCO format:
        [top left x position, top left y position, width, height]

    Returns
    -------
    list[float]
        bounding box annotation in YOLO format:
        [x_center_rel, y_center_rel, width_rel, height_rel]
    """

    # YOLO bounding box format: [x_center, y_center, width, height]
    # (float values relative to width and height of image)
    x_tl, y_tl, w, h = bbox

    dw = 1.0 / img_width
    dh = 1.0 / img_height

    x_center = x_tl + w / 2.0
    y_center = y_tl + h / 2.0

    x = x_center * dw
    y = y_center * dh
    w = w * dw
    h = h * dh

    return [x, y, w, h]

def convert_coco_json_to_yolo_txt(output_path, json_file):

    path = output_path#make_folders(output_path)

    with open(json_file) as f:
        json_data = json.load(f)

    # write _darknet.labels, which holds names of all classes (one class per line)
    label_file = os.path.join(output_path, "_darknet.labels")
    with open(label_file, "w") as f:
        for category in tqdm(json_data["categories"], desc="Categories"):
            category_name = category["name"]
            f.write(f"{category_name}\n")

    for image in tqdm(json_data["images"], desc="Annotation txt for each iamge"):
        img_id = image["id"]
        img_name = image["file_name"]
        img_width = image["width"]
        img_height = image["height"]

        anno_in_image = [anno for anno in json_data["annotations"] if anno["image_id"] == img_id]
        anno_txt = os.path.join(output_path, img_name.split(".")[0] + ".txt")
        with open(anno_txt, "w") as f:
            for anno in anno_in_image:
                category = anno["category_id"]
                bbox_COCO = anno["bbox"]
                x, y, w, h = convert_bbox_coco2yolo(img_width, img_height, bbox_COCO)
                f.write(f"{category} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

    print("Converting COCO Json to YOLO txt finished!")


In [ ]:
#move the file thermal_annotations.json to FLIR/train
src = 'thermal_annotations.json'
dst = 'datasets/FLIR/train'
shutil.move(src, dst)

'FLIR/train/thermal_annotations.json'

In [28]:
coco_format_json = "FLIR/train/thermal_annotations.json"
output_path = "FLIR/train/"

convert_coco_json_to_yolo_txt(output_path, coco_format_json)

Annotation txt for each iamge: 100%|██████████| 8862/8862 [01:36<00:00, 91.64it/s]

Converting COCO Json to YOLO txt finished!


In [30]:
!nvidia-smi

Fri Nov 15 01:05:41 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.05             Driver Version: 550.127.05     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX TITAN X     Off |   00000000:03:00.0  On |                  N/A |
| 22%   41C    P8             17W /  250W |     293MiB /  12288MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [31]:
!pip3 install torch torchvision torchaudio

     |████████████████████████████████| 3.4 MB 2.8 MB/s eta 0:00:01


In [4]:
import json

json_path = 'datasets/FLIR/train/thermal_annotations.json'
json_out_path = json_path
# Carica il file delle annotazioni
with open(json_path, 'r') as f:
    data = json.load(f)

# Aggiorna category_id nelle annotazioni
for annotation in data['annotations']:
    old_id = annotation['category_id']
    annotation['category_id'] = old_id - 1

# Aggiorna categories nel file
for category in data['categories']:
    old_id = category['id']
    category['id'] = old_id - 1

# Salva il file aggiornato
with open(json_out_path, 'w') as f:
    json.dump(data, f)


In [16]:
#in all txt files in FLIR/train, subtract 1 from the first number in each line
import os

path = 'datasets/FLIR/train/'
for file in os.listdir(path):
    if file.endswith('.txt'):
        with open(os.path.join(path, file), 'r') as f:
            lines = f.readlines()
        with open(os.path.join(path, file), 'w') as f:
            for line in lines:
                f.write(str(int(line[0])-1) + line[1:])


In [ ]:
#verify if the GPU is available
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
0
2
NVIDIA GeForce GTX TITAN X


In [ ]:
from ultralytics import YOLO
model = YOLO("runs/detect/train/weights/best.pt")#("yolo11m.pt")  # load a pretrained model (recommended for training)
results = model.train(data="FLIR.yaml", epochs=50, imgsz=640, save=True, patience=5, time=24, batch=0.90, save_period=1, pretrained=True, freeze=None, plots=True)

Ultralytics 8.3.31 🚀 Python-3.8.10 torch-2.4.1+cu121 CUDA:3 (NVIDIA GeForce GTX TITAN X, 12200MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=FLIR.yaml, epochs=600, time=None, patience=100, batch=0.95, imgsz=640, save=True, save_period=-1, cache=disk, device=3, workers=8, project=YOLO11, name=yolo11m10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=yolo11m.pt, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, 

train: Scanning /homes/students_home/davidmegli/yolo-finetuning/datasets/FLIR/train.cache... 8862 images, 1002 backgrounds, 202 corrupt: 100%|██████████| 8862/8862 [00:00<?, ?it/s]

train: WARNING ⚠️ /homes/students_home/davidmegli/yolo-finetuning/datasets/FLIR/train/FLIR_01059.jpeg: ignoring corrupt image/label: Label class 17 exceeds dataset class count 10. Possible class labels are 0-9
train: WARNING ⚠️ /homes/students_home/davidmegli/yolo-finetuning/datasets/FLIR/train/FLIR_01271.jpeg: ignoring corrupt image/label: Label class 17 exceeds dataset class count 10. Possible class labels are 0-9
train: WARNING ⚠️ /homes/students_home/davidmegli/yolo-finetuning/datasets/FLIR/train/FLIR_02091.jpeg: ignoring corrupt image/label: Label class 17 exceeds dataset class count 10. Possible class labels are 0-9
train: WARNING ⚠️ /homes/students_home/davidmegli/yolo-finetuning/datasets/FLIR/train/FLIR_02094.jpeg: ignoring corrupt image/label: Label class 17 exceeds dataset class count 10. Possible class labels are 0-9
train: WARNING ⚠️ /homes/students_home/davidmegli/yolo-finetuning/datasets/FLIR/train/FLIR_02095.jpeg: ignoring corrupt image/label: Label class 17 exceeds data


train: Caching images (7.9GB Disk): 100%|██████████| 8660/8660 [00:00<00:00, 24169.37it/s]


AutoBatch: Computing optimal batch size for imgsz=640 at 95.0% CUDA memory utilization.
AutoBatch: CUDA:3 (NVIDIA GeForce GTX TITAN X) 11.91G total, 0.20G reserved, 0.18G allocated, 11.53G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
    20115459       68.53         0.801         49.54         111.3        (1, 3, 640, 640)                    list
    20115459       137.1         1.309         51.45         122.6        (2, 3, 640, 640)                    list
    20115459       274.1         2.571         92.05         173.9        (4, 3, 640, 640)                    list
    20115459       548.3         4.706         169.4         287.9        (8, 3, 640, 640)                    list
    20115459        1097         8.605         322.8         514.9       (16, 3, 640, 640)                    list
AutoBatch: Using batch-size 20 for CUDA:3 11.17G/11.91G (94%) ✅


...


train: Caching images (7.9GB Disk): 100%|██████████| 8660/8660 [00:00<00:00, 23415.05it/s]
val: Scanning /homes/students_home/davidmegli/yolo-finetuning/datasets/FLIR/val.cache... 1359 images, 7 backgrounds, 0 corrupt: 100%|██████████| 1366/1366 [00:00<?, ?it/s]

val: WARNING ⚠️ /homes/students_home/davidmegli/yolo-finetuning/datasets/FLIR/val/FLIR_09042.jpeg: 1 duplicate labels removed
val: WARNING ⚠️ /homes/students_home/davidmegli/yolo-finetuning/datasets/FLIR/val/FLIR_09055.jpeg: 1 duplicate labels removed
val: WARNING ⚠️ /homes/students_home/davidmegli/yolo-finetuning/datasets/FLIR/val/FLIR_09653.jpeg: 1 duplicate labels removed



val: Caching images (1.3GB Disk): 100%|██████████| 1366/1366 [00:00<00:00, 23797.62it/s]


Plotting labels to YOLO11/yolo11m10/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 106 weight(decay=0.0), 113 weight(decay=0.00046875), 112 bias(decay=0.0)


AssertionError: yolo11m.pt training to 600 epochs is finished, nothing to resume.
Start a new training without resuming, i.e. 'yolo train model=yolo11m.pt'

In [2]:
import matplotlib.pyplot as plt
import pandas as pd

# Path al file di logging degli esperimenti
log_dir = 'runs/detect/train/results.csv'

# Carica i dati dal file CSV
data = pd.read_csv(log_dir)

# Grafico della loss
plt.figure(figsize=(12, 6))
plt.plot(data['epoch'], data['loss'], label='Loss', color='blue')
plt.plot(data['epoch'], data['val_loss'], label='Validation Loss', color='orange')
plt.title('Loss vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()

# Grafico della mAP (mean Average Precision)
plt.figure(figsize=(12, 6))
plt.plot(data['epoch'], data['maps'], label='mAP', color='green')
plt.title('mAP vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('mAP')
plt.legend()
plt.grid()
plt.show()


KeyError: 'loss'

<Figure size 1200x600 with 0 Axes>